In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
import kagglehub
kmader_food41_path = kagglehub.dataset_download('kmader/food41')

print('Data source import complete.')


# CNN ON FOOD-101 DATASET

I am Rishabh Jain and this is my attempt at classifying the foods in the food-101 dataset

### STEP 1: Importing the Libraries

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pickle

### STEP 2: Correcting the Dataset

The dataset we have used as input conatins the labels separate from the images so we have to do an extra step which makes a new dataset with the food name as the folder name of the images

In [ ]:
import os

def make_new_data(txt_file, s_folder, d_folder):
    os.makedirs(d_folder, exist_ok=True)
    with open(txt_file, 'r') as f:
        for line in f:
            full_path = line.strip()
            food_name = full_path.split('/')[0]
            dst_folder = os.path.join(d_folder, food_name)
            os.makedirs(dst_folder, exist_ok=True)
            src_path = os.path.join(s_folder, full_path + '.jpg')
            dst_path = os.path.join(dst_folder, os.path.basename(full_path) + '.jpg')
            try:
                os.symlink(src_path, dst_path)
            except FileExistsError:
                pass

make_new_data('/kaggle/input/food41/meta/meta/train.txt', '/kaggle/input/food41/images', 'train')
make_new_data('/kaggle/input/food41/meta/meta/test.txt', '/kaggle/input/food41/images', 'test')

### STEP 3: Getting the data, processing it and loading it

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

#transform functions composed
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])

#getting the dataset
train_data = ImageFolder(root = "train" , transform = transform)
test_data = ImageFolder(root = "test" , transform = transform)
train_size = len(train_data)
test_size = len(test_data)

#loading data
train_loader = DataLoader(train_data, batch_size=64, shuffle=True,num_workers=os.cpu_count(),pin_memory=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False,num_workers=os.cpu_count(),pin_memory=True)


### STEP 4: Making the Model

In [ ]:
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,3,1,1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,3,1,1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(64,128,3,1,1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(128,256,3,1,1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu = nn.ReLU()
        self.pool4 = nn.MaxPool2d(2,2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256*14*14,1024)
        self.dp1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024,512)
        self.dp2 = nn.Dropout(0.5)
        self.out = nn.Linear(512,101)

    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.pool4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dp1(x)
        x = self.fc2(x)
        x = self.dp2(x)
        return self.out(x)

### STEP 5: Setting the optimizer and the loss function

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr = 0.001)
lossfn = nn.CrossEntropyLoss()

### STEP 6: Main code of the model

In [ ]:
def train(model, train_loader, lossfn, optimizer, num_epochs):
    train_losses = []
    train_accs = []

    for epoch in range(num_epochs):
        model.train()
        trainloss = 0.0
        traincorrect = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = lossfn(outputs, labels)
            loss.backward()
            optimizer.step()
            trainloss += loss.item()*inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            traincorrect += (predicted == labels).sum().item()

        avgtrainloss = trainloss / train_size
        trainacc = traincorrect / train_size
        train_losses.append(avgtrainloss)
        train_accs.append(trainacc)
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avgtrainloss:.4f} , Train Acc: {trainacc:.4f}")

    return train_losses, train_accs

### STEP 6: Training the model and saving the weights

In [ ]:
train_losses, train_accs = train(model,train_loader,lossfn,optimizer,num_epochs=15)
with open("food_101_model.pkl", "wb") as f:
    pickle.dump(model.state_dict(), f)

### STEP 7: Testing the model and saving predictions in submissions.csv

In [ ]:
def testmodel(model, testloader):
    model.eval()
    preds = []
    indices = []

    with torch.no_grad():
        for batch_id, (inputs, _) in enumerate(testloader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            indices.extend(range(batch_id * testloader.batch_size, batch_id * testloader.batch_size + inputs.size(0)))

    df = pd.DataFrame({'Id': indices,'Predicted': preds})
    df.to_csv("submissions.csv", index=False)

testmodel(model,test_loader)


### STEP 8: Plotting train loss and accuracy plots

In [ ]:
def plot_training_history(train_losses, train_accs):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(12,5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'b-', label='Train Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accs, 'b-', label='Train Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Train Accuracy')
    plt.legend()

    plt.show()

plot_training_history(train_losses, train_accs)

### STEP 9: Show predictions of some images

In [ ]:
import random

def imageshow(img, title):
    img = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()

class_names = test_data.classes
def show_top5_preds(model, testloader, classnames, num_images):
    model.eval()
    indices = random.sample(range(len(testloader)), num_images)
    with torch.no_grad():
        for id in indices:
            image,_ = test_data[id]
            inputs = image.unsqueeze(0).to(device)
            probs = torch.softmax(model(inputs),dim = 1)
            top5_probs, top5_indices = torch.topk(probs, k = 5, dim = 1)
            top5 = zip(top5_indices[0].cpu().numpy(), top5_probs[0].cpu().numpy())
            top5_str = "\n".join([f"{classnames[class_id]}: {prob:.3f}" for class_id, prob in top5])
            imageshow(image, top5_str)

show_top5_preds(model,test_loader, class_names, num_images = 2)